# Module 12: Clustering - K-Means and DBSCAN

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 75 minutes  
**Prerequisites**: [Module 01 - Supervised vs Unsupervised Learning](01_supervised_vs_unsupervised_learning.ipynb), [Module 02 - Data Preparation](02_data_preparation_train_test_split.ipynb)

## Learning Objectives
By the end of this notebook, you will be able to:
1. Understand unsupervised learning and clustering concepts
2. Implement K-Means clustering and understand how it works
3. Use the elbow method to choose the optimal number of clusters (K)
4. Evaluate clustering quality with silhouette score and inertia
5. Recognize K-Means limitations (spherical clusters, sensitivity to initialization)
6. Apply DBSCAN for density-based clustering
7. Tune DBSCAN parameters (eps and min_samples)
8. Compare K-Means and DBSCAN on different data shapes
9. Know when to use each clustering algorithm

## 1. Introduction: What is Clustering?

### Unsupervised Learning Refresher

**Supervised Learning** (what we've done so far):
- We have labels (target variable)
- Goal: Predict labels for new data
- Examples: Classification, regression

**Unsupervised Learning** (clustering):
- We DON'T have labels
- Goal: Find patterns, structure, or groupings in data
- Examples: Clustering, dimensionality reduction

### What is Clustering?

**Clustering** is the task of grouping similar data points together.

**Real-world examples:**
- **Customer segmentation**: Group customers by buying behavior
- **Document organization**: Group similar articles or papers
- **Image segmentation**: Group pixels into regions
- **Anomaly detection**: Points that don't fit any cluster
- **Gene expression analysis**: Group genes with similar patterns

### Key Questions in Clustering

1. **How many clusters?** (K in K-Means)
2. **What makes points similar?** (Distance metric)
3. **How to evaluate quality?** (No labels to check against!)
4. **What shape are clusters?** (Spherical, arbitrary, density-based)

## 2. Setup and Data Loading

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
import warnings

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)
%matplotlib inline

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ All libraries imported successfully!')
print(f'✓ Random seed set to 42 for reproducibility')

In [ ]:
# Load clustering datasets
# Dataset 1: Blob-like clusters (perfect for K-Means)
blobs_df = pd.read_csv('data/sample/blobs_clustering.csv')

print("Blobs Dataset (Well-separated spherical clusters):")
print(f"Shape: {blobs_df.shape}")
print(f"\nFirst few rows:")
print(blobs_df.head())

# Dataset 2: Moon-shaped clusters (challenging for K-Means, good for DBSCAN)
moons_df = pd.read_csv('data/sample/moons_nonlinear.csv')

print("\n" + "="*50)
print("\nMoons Dataset (Non-linear, crescent-shaped clusters):")
print(f"Shape: {moons_df.shape}")
print(f"\nFirst few rows:")
print(moons_df.head())

In [ ]:
# Visualize both datasets
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Blobs dataset
axes[0].scatter(blobs_df['feature_1'], blobs_df['feature_2'], alpha=0.6, s=50)
axes[0].set_xlabel('Feature 1', fontsize=12)
axes[0].set_ylabel('Feature 2', fontsize=12)
axes[0].set_title('Blobs Dataset\n(Spherical clusters)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Moons dataset
axes[1].scatter(moons_df['feature_1'], moons_df['feature_2'], alpha=0.6, s=50)
axes[1].set_xlabel('Feature 1', fontsize=12)
axes[1].set_ylabel('Feature 2', fontsize=12)
axes[1].set_title('Moons Dataset\n(Non-linear clusters)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Notice:")
print("- Left: Clear, spherical clusters - perfect for K-Means!")
print("- Right: Crescent shapes - challenging for K-Means, better for DBSCAN!")

## 3. K-Means Clustering: The Algorithm

### How K-Means Works

K-Means is an iterative algorithm that partitions data into K clusters:

**Step 1: Initialization**
- Choose K (number of clusters)
- Randomly place K cluster centers (centroids)

**Step 2: Assignment**
- Assign each point to the nearest centroid
- Forms K clusters

**Step 3: Update**
- Move each centroid to the mean of its assigned points
- Centroids shift to cluster centers

**Step 4: Repeat**
- Repeat steps 2-3 until centroids stop moving (convergence)
- Or reach maximum iterations

### Objective Function

K-Means minimizes **within-cluster sum of squares (WCSS)** or **inertia**:

$$\text{Inertia} = \sum_{i=1}^{K} \sum_{x \in C_i} ||x - \mu_i||^2$$

Where:
- K = number of clusters
- C_i = points in cluster i
- μ_i = centroid of cluster i
- || || = Euclidean distance

**In words**: Sum of squared distances from each point to its cluster center

In [ ]:
# Apply K-Means to blobs dataset
# First, let's try K=3 (we can see 3 clusters visually)

X_blobs = blobs_df[['feature_1', 'feature_2']].values

# Scale features (important for K-Means!)
scaler = StandardScaler()
X_blobs_scaled = scaler.fit_transform(X_blobs)

# Create and fit K-Means model
kmeans = KMeans(
    n_clusters=3,           # Number of clusters
    init='k-means++',       # Smart initialization (better than random)
    n_init=10,              # Number of times to run with different initializations
    max_iter=300,           # Maximum iterations
    random_state=42
)

# Fit and predict cluster labels
cluster_labels = kmeans.fit_predict(X_blobs_scaled)

# Get cluster centers
centers = kmeans.cluster_centers_

# Get inertia (lower is better - tighter clusters)
inertia = kmeans.inertia_

print("K-Means Clustering Results (K=3):")
print("=" * 50)
print(f"Number of iterations: {kmeans.n_iter_}")
print(f"Inertia (WCSS): {inertia:.2f}")
print(f"\nCluster centers (scaled coordinates):")
print(centers)
print(f"\nCluster sizes:")
unique, counts = np.unique(cluster_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} points")

In [ ]:
# Visualize K-Means results
plt.figure(figsize=(12, 5))

# Plot 1: Original data
plt.subplot(1, 2, 1)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], alpha=0.6, s=50, c='gray')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Before Clustering', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot 2: Clustered data
plt.subplot(1, 2, 2)
# Plot points colored by cluster
scatter = plt.scatter(
    X_blobs[:, 0], X_blobs[:, 1],
    c=cluster_labels,
    cmap='viridis',
    alpha=0.6,
    s=50
)

# Plot cluster centers
centers_original = scaler.inverse_transform(centers)
plt.scatter(
    centers_original[:, 0], centers_original[:, 1],
    c='red',
    marker='X',
    s=300,
    edgecolors='black',
    linewidth=2,
    label='Centroids'
)

plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('After K-Means Clustering (K=3)', fontsize=14, fontweight='bold')
plt.legend()
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Perfect! K-Means correctly identified the 3 spherical clusters.")

## 4. Choosing K: The Elbow Method

### The Challenge

**Problem**: How do we choose K (number of clusters)?
- Too few clusters: Underfitting, miss patterns
- Too many clusters: Overfitting, meaningless groups

### The Elbow Method

**Idea**: Plot inertia (WCSS) vs K
- As K increases, inertia decreases (points get closer to centroids)
- Eventually, gains diminish (diminishing returns)
- Choose K at the "elbow" - where the curve bends sharply

**Interpretation**:
- Sharp drop before elbow: Adding clusters helps a lot
- Gradual decline after elbow: Adding clusters doesn't help much
- Elbow point: Optimal tradeoff

In [ ]:
# Apply elbow method: Test K from 1 to 10
K_range = range(1, 11)
inertias = []
silhouette_scores = []

print("Testing different values of K...\n")

for k in K_range:
    # Train K-Means with k clusters
    kmeans_test = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    kmeans_test.fit(X_blobs_scaled)
    
    # Store inertia
    inertias.append(kmeans_test.inertia_)
    
    # Calculate silhouette score (requires at least 2 clusters)
    if k > 1:
        labels = kmeans_test.labels_
        silhouette_avg = silhouette_score(X_blobs_scaled, labels)
        silhouette_scores.append(silhouette_avg)
    else:
        silhouette_scores.append(0)
    
    print(f"K={k:2d} | Inertia: {kmeans_test.inertia_:7.2f} | Silhouette: {silhouette_scores[-1]:.3f}")

print("\n✓ Testing complete!")

In [ ]:
# Visualize the elbow method
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Elbow curve (Inertia)
axes[0].plot(K_range, inertias, 'o-', linewidth=2, markersize=8)
axes[0].axvline(3, color='red', linestyle='--', label='Optimal K=3', alpha=0.7)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia (WCSS)', fontsize=12)
axes[0].set_title('Elbow Method: Inertia vs K', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Add annotation for elbow
axes[0].annotate('Elbow Point', xy=(3, inertias[2]), xytext=(5, inertias[2] + 100),
                arrowprops=dict(arrowstyle='->', color='red', lw=2),
                fontsize=11, color='red')

# Plot 2: Silhouette score
axes[1].plot(K_range, silhouette_scores, 's-', linewidth=2, markersize=8, color='green')
axes[1].axvline(3, color='red', linestyle='--', label='Optimal K=3', alpha=0.7)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score vs K', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("- Left: Inertia drops sharply until K=3, then levels off (elbow at K=3)")
print("- Right: Silhouette score is highest at K=3 (best cluster separation)")
print("\n✅ Both methods agree: K=3 is optimal!")

## 5. Evaluating Cluster Quality: Silhouette Score

### What is Silhouette Score?

**Measures how well each point fits its cluster** compared to other clusters.

For each point:
- **a**: Mean distance to other points in same cluster (cohesion)
- **b**: Mean distance to points in nearest different cluster (separation)
- **Silhouette coefficient**: (b - a) / max(a, b)

**Range**: -1 to +1
- **+1**: Point is far from other clusters, close to its own (perfect)
- **0**: Point is on the border between clusters
- **-1**: Point might be in wrong cluster

**Average silhouette score**: Mean across all points
- **> 0.7**: Strong clustering structure
- **0.5 - 0.7**: Reasonable structure
- **0.25 - 0.5**: Weak structure
- **< 0.25**: No substantial structure

In [ ]:
# Calculate detailed silhouette scores
silhouette_vals = silhouette_samples(X_blobs_scaled, cluster_labels)
silhouette_avg = silhouette_score(X_blobs_scaled, cluster_labels)

print("Silhouette Analysis:")
print("=" * 50)
print(f"Average Silhouette Score: {silhouette_avg:.3f}")
print("\nPer-cluster Silhouette Scores:")

for cluster in range(3):
    cluster_silhouette_vals = silhouette_vals[cluster_labels == cluster]
    print(f"  Cluster {cluster}: {cluster_silhouette_vals.mean():.3f} (n={len(cluster_silhouette_vals)})")

print("\n✅ Score > 0.7 indicates strong, well-separated clusters!")

In [ ]:
# Visualize silhouette plot
fig, ax = plt.subplots(figsize=(10, 6))

y_lower = 10
for cluster in range(3):
    # Get silhouette values for this cluster
    cluster_silhouette_vals = silhouette_vals[cluster_labels == cluster]
    cluster_silhouette_vals.sort()
    
    size_cluster = len(cluster_silhouette_vals)
    y_upper = y_lower + size_cluster
    
    color = plt.cm.viridis(float(cluster) / 3)
    ax.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        cluster_silhouette_vals,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
        label=f'Cluster {cluster}'
    )
    
    # Label the clusters
    ax.text(-0.05, y_lower + 0.5 * size_cluster, str(cluster), fontsize=12)
    
    y_lower = y_upper + 10

# Add average line
ax.axvline(silhouette_avg, color='red', linestyle='--', linewidth=2, label=f'Average: {silhouette_avg:.3f}')

ax.set_xlabel('Silhouette Coefficient', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)
ax.set_title('Silhouette Plot for K-Means Clustering', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("- Width of each colored region: Number of points in cluster")
print("- Thickness: How far points extend to the right (higher = better fit)")
print("- All clusters exceed average line: All clusters are well-formed!")

## 6. K-Means Limitations

### When K-Means Struggles

**1. Non-spherical clusters**
- K-Means assumes spherical clusters
- Fails on elongated, crescent, or irregular shapes

**2. Different sized clusters**
- K-Means tries to make equal-sized clusters
- Struggles when clusters have very different sizes

**3. Different densities**
- K-Means assumes similar density
- Fails when some clusters are dense, others sparse

**4. Sensitive to initialization**
- Random initialization can lead to different results
- Solution: Use k-means++ initialization, run multiple times

**5. Requires K in advance**
- Must specify number of clusters beforehand
- Not always obvious what K should be

**6. Outliers affect centroids**
- Outliers pull centroids away from true centers
- Sensitive to noise

Let's demonstrate these limitations:

In [ ]:
# Demonstrate K-Means failure on non-spherical data (moons)
X_moons = moons_df[['feature_1', 'feature_2']].values

# Scale features
scaler_moons = StandardScaler()
X_moons_scaled = scaler_moons.fit_transform(X_moons)

# Apply K-Means with K=2 (we can see 2 crescent shapes)
kmeans_moons = KMeans(n_clusters=2, init='k-means++', n_init=10, random_state=42)
labels_moons = kmeans_moons.fit_predict(X_moons_scaled)

# Calculate silhouette score
silhouette_moons = silhouette_score(X_moons_scaled, labels_moons)

# Visualize K-Means on moons
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_moons[:, 0], X_moons[:, 1], alpha=0.6, s=50, c='gray')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Moons Dataset\n(Two crescent-shaped clusters)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
scatter = plt.scatter(X_moons[:, 0], X_moons[:, 1], c=labels_moons, cmap='viridis', alpha=0.6, s=50)
centers_moons = scaler_moons.inverse_transform(kmeans_moons.cluster_centers_)
plt.scatter(centers_moons[:, 0], centers_moons[:, 1], c='red', marker='X', s=300, 
           edgecolors='black', linewidth=2, label='Centroids')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title(f'K-Means Result (K=2)\nSilhouette: {silhouette_moons:.3f}', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n❌ K-Means Failure:")
print(f"   Silhouette Score: {silhouette_moons:.3f} (poor!)")
print("   K-Means splits each crescent in half instead of separating the crescents.")
print("   This is because K-Means assumes spherical clusters!")
print("\n💡 Solution: Use density-based clustering like DBSCAN!")

## 7. DBSCAN: Density-Based Clustering

### What is DBSCAN?

**DBSCAN** = Density-Based Spatial Clustering of Applications with Noise

**Key idea**: Clusters are dense regions separated by sparse regions

### How DBSCAN Works

**Core concepts:**
1. **Core point**: Has at least min_samples neighbors within distance eps
2. **Border point**: Within eps of a core point, but not a core point itself
3. **Noise point**: Neither core nor border (outlier)

**Algorithm:**
1. For each point, count neighbors within distance eps
2. Mark as core point if it has ≥ min_samples neighbors
3. Form clusters by connecting core points that are neighbors
4. Add border points to nearby clusters
5. Mark remaining points as noise (cluster label = -1)

### Parameters

**eps (epsilon)**:
- Maximum distance between two points to be neighbors
- Larger eps → fewer, larger clusters
- Smaller eps → more, smaller clusters (more noise)

**min_samples**:
- Minimum points needed to form a dense region (core point)
- Larger min_samples → fewer core points, more noise
- Rule of thumb: min_samples = 2 × dimensions

### Advantages of DBSCAN

✅ **No need to specify K** (number of clusters)  
✅ **Handles arbitrary shapes** (non-spherical clusters)  
✅ **Robust to outliers** (labels them as noise)  
✅ **Handles different densities** (to some extent)  
✅ **Deterministic** (same result every run)

In [ ]:
# Apply DBSCAN to moons dataset
# Start with reasonable parameters
dbscan = DBSCAN(
    eps=0.3,              # Maximum distance between neighbors
    min_samples=5         # Minimum points to form a dense region
)

# Fit and predict cluster labels
labels_dbscan = dbscan.fit_predict(X_moons_scaled)

# Count clusters (excluding noise points labeled -1)
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_noise = list(labels_dbscan).count(-1)

print("DBSCAN Clustering Results:")
print("=" * 50)
print(f"Parameters: eps={dbscan.eps}, min_samples={dbscan.min_samples}")
print(f"\nNumber of clusters found: {n_clusters}")
print(f"Number of noise points: {n_noise}")
print(f"\nCluster sizes:")
unique, counts = np.unique(labels_dbscan, return_counts=True)
for cluster, count in zip(unique, counts):
    if cluster == -1:
        print(f"  Noise: {count} points")
    else:
        print(f"  Cluster {cluster}: {count} points")

# Calculate silhouette score (excluding noise points)
if n_clusters > 1 and n_noise < len(labels_dbscan):
    mask = labels_dbscan != -1
    silhouette_dbscan = silhouette_score(X_moons_scaled[mask], labels_dbscan[mask])
    print(f"\nSilhouette Score (excluding noise): {silhouette_dbscan:.3f}")

In [ ]:
# Visualize DBSCAN results
plt.figure(figsize=(12, 5))

# Plot 1: K-Means (poor result)
plt.subplot(1, 2, 1)
scatter1 = plt.scatter(X_moons[:, 0], X_moons[:, 1], c=labels_moons, cmap='viridis', alpha=0.6, s=50)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title(f'K-Means (K=2)\nSilhouette: {silhouette_moons:.3f}', fontsize=14, fontweight='bold')
plt.colorbar(scatter1, label='Cluster')
plt.grid(True, alpha=0.3)

# Plot 2: DBSCAN (excellent result)
plt.subplot(1, 2, 2)
# Plot noise points in gray
noise_mask = labels_dbscan == -1
plt.scatter(X_moons[noise_mask, 0], X_moons[noise_mask, 1], 
           c='gray', alpha=0.3, s=50, label='Noise')
# Plot clustered points
plt.scatter(X_moons[~noise_mask, 0], X_moons[~noise_mask, 1], 
           c=labels_dbscan[~noise_mask], cmap='viridis', alpha=0.6, s=50)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title(f'DBSCAN (eps={dbscan.eps}, min_samples={dbscan.min_samples})\nSilhouette: {silhouette_dbscan:.3f}', 
         fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ DBSCAN Success:")
print("   Correctly identified the two crescent-shaped clusters!")
print("   Much better silhouette score than K-Means.")
print("   DBSCAN can handle non-spherical cluster shapes.")

## 8. Tuning DBSCAN Parameters

### Choosing eps (epsilon)

**Method**: K-distance plot
1. For each point, calculate distance to its K-th nearest neighbor
2. Sort these distances in ascending order
3. Plot the sorted distances
4. Look for "elbow" - sharp increase indicates noise
5. Choose eps at the elbow

**Rules of thumb**:
- Start with eps = distance to 4th or 5th nearest neighbor
- Too small → many small clusters and noise
- Too large → everything in one cluster

In [ ]:
# K-distance plot to find optimal eps
from sklearn.neighbors import NearestNeighbors

# Calculate distances to 5th nearest neighbor for each point
k = 5
neighbors = NearestNeighbors(n_neighbors=k)
neighbors.fit(X_moons_scaled)
distances, indices = neighbors.kneighbors(X_moons_scaled)

# Get distance to k-th nearest neighbor for each point
k_distances = distances[:, k-1]
k_distances_sorted = np.sort(k_distances)

# Plot k-distance graph
plt.figure(figsize=(10, 6))
plt.plot(k_distances_sorted, linewidth=2)
plt.xlabel('Points sorted by distance', fontsize=12)
plt.ylabel(f'Distance to {k}-th nearest neighbor', fontsize=12)
plt.title('K-Distance Plot for Finding Optimal eps', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Add horizontal line at elbow
optimal_eps = 0.3
plt.axhline(optimal_eps, color='red', linestyle='--', label=f'Suggested eps={optimal_eps}', linewidth=2)
plt.legend()

plt.tight_layout()
plt.show()

print("\n💡 How to read this plot:")
print("- Flat region: Dense areas (points close together)")
print("- Sharp increase: Transition to sparse areas/noise")
print(f"- Elbow point around {optimal_eps}: Good choice for eps")
print("\n✅ Choose eps where the curve starts to rise sharply!")

In [ ]:
# Compare different eps values
eps_values = [0.1, 0.2, 0.3, 0.5]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, eps in enumerate(eps_values):
    # Apply DBSCAN with this eps
    dbscan_test = DBSCAN(eps=eps, min_samples=5)
    labels_test = dbscan_test.fit_predict(X_moons_scaled)
    
    # Count clusters and noise
    n_clusters_test = len(set(labels_test)) - (1 if -1 in labels_test else 0)
    n_noise_test = list(labels_test).count(-1)
    
    # Calculate silhouette if possible
    if n_clusters_test > 1 and n_noise_test < len(labels_test):
        mask = labels_test != -1
        silhouette_test = silhouette_score(X_moons_scaled[mask], labels_test[mask])
    else:
        silhouette_test = -1
    
    # Plot
    noise_mask_test = labels_test == -1
    axes[idx].scatter(X_moons[noise_mask_test, 0], X_moons[noise_mask_test, 1],
                     c='gray', alpha=0.3, s=30, label='Noise')
    axes[idx].scatter(X_moons[~noise_mask_test, 0], X_moons[~noise_mask_test, 1],
                     c=labels_test[~noise_mask_test], cmap='viridis', alpha=0.6, s=30)
    axes[idx].set_xlabel('Feature 1', fontsize=10)
    axes[idx].set_ylabel('Feature 2', fontsize=10)
    axes[idx].set_title(f'eps={eps}\nClusters: {n_clusters_test}, Noise: {n_noise_test}\nSilhouette: {silhouette_test:.3f}',
                       fontsize=12, fontweight='bold')
    axes[idx].legend(loc='upper right', fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Effect of eps:")
print("- eps=0.1: Too small → many clusters, lots of noise")
print("- eps=0.2: Better, but still some fragmentation")
print("- eps=0.3: ✅ Optimal - correctly identifies 2 clusters")
print("- eps=0.5: Too large → merges clusters together")

## 9. K-Means vs DBSCAN: When to Use Each

### Use K-Means When:

✅ **Clusters are roughly spherical** (circular, blob-like)  
✅ **Similar cluster sizes** (roughly equal number of points)  
✅ **Similar densities** across clusters  
✅ **Know K in advance** (or can estimate it)  
✅ **Large datasets** (K-Means is very fast)  
✅ **Want guaranteed K clusters** (no noise points)

**Examples**: Customer segmentation, image compression, document clustering

### Use DBSCAN When:

✅ **Arbitrary cluster shapes** (crescents, elongated, irregular)  
✅ **Don't know K** (number of clusters)  
✅ **Varying cluster sizes** (some big, some small)  
✅ **Varying densities** (some dense, some sparse)  
✅ **Want to detect outliers** (noise points)  
✅ **Spatial/geographic data** (location-based clustering)

**Examples**: Anomaly detection, geographic data, image segmentation, astronomy

### Summary Comparison Table

| Feature | K-Means | DBSCAN |
|---------|---------|--------|
| Cluster shape | Spherical only | Arbitrary |
| Specify K? | Yes | No |
| Handles outliers? | No (assigns to nearest) | Yes (marks as noise) |
| Speed | Very fast | Slower |
| Deterministic? | No (depends on init) | Yes |
| Different densities? | No | Somewhat |
| Different sizes? | No | Yes |

## Exercises

Now it's your turn to practice! Complete these exercises to reinforce your understanding.

### Exercise 1: K-Means on Iris Dataset

Apply K-Means clustering to the Iris dataset (without using species labels).

**Tasks:**
1. Load `iris.csv` and extract only the feature columns
2. Scale the features
3. Use the elbow method to find optimal K (test K from 2 to 8)
4. Apply K-Means with optimal K
5. Calculate silhouette score
6. Visualize the clusters using the first 2 features
7. Compare your clusters to the true species labels (how well did it do?)

In [ ]:
# Your code here
# Hint: Follow the K-Means workflow from sections 3-5



### Exercise 2: DBSCAN Parameter Tuning

Find optimal DBSCAN parameters for the blobs dataset.

**Tasks:**
1. Use the blobs dataset
2. Create a K-distance plot to estimate optimal eps
3. Test different combinations of eps (0.3, 0.5, 0.7, 1.0) and min_samples (3, 5, 10)
4. For each combination, count clusters and noise points
5. Calculate silhouette scores
6. Create a heatmap showing silhouette scores for each parameter combination
7. Which combination works best?

In [ ]:
# Your code here
# Hint: Use nested loops to test all combinations



### Exercise 3: Comparing Algorithms on Wine Dataset

Compare K-Means and DBSCAN on the wine dataset.

**Tasks:**
1. Load `wine.csv` (drop target column for unsupervised learning)
2. Scale features
3. Apply K-Means with K=3
4. Apply DBSCAN with reasonable parameters
5. Calculate silhouette scores for both
6. Calculate Davies-Bouldin score for both (lower is better)
7. Visualize both results using first 2 features
8. Which algorithm performs better? Why?

In [ ]:
# Your code here
# Hint: from sklearn.metrics import davies_bouldin_score



### Exercise 4: Outlier Detection with DBSCAN

Use DBSCAN to detect outliers in the breast cancer dataset.

**Tasks:**
1. Load `breast_cancer.csv` (drop target column)
2. Scale features
3. Apply DBSCAN with eps=0.5, min_samples=10
4. How many outliers (noise points) were detected?
5. What percentage of total data is considered noise?
6. Visualize using first 2 PCA components (hint: use PCA for dimensionality reduction)
7. Try different eps values (0.3, 0.5, 1.0) - how does it affect outlier detection?

In [ ]:
# Your code here
# Hint: Outliers have cluster label = -1
# For visualization: from sklearn.decomposition import PCA



## Summary

### Key Concepts Learned

1. **Clustering Fundamentals**
   - Unsupervised learning: finding patterns without labels
   - Goal: Group similar points together
   - Applications: segmentation, organization, anomaly detection

2. **K-Means Clustering**
   - Iterative algorithm: assign → update → repeat
   - Minimizes within-cluster sum of squares (inertia)
   - Fast and scalable
   - Best for spherical, similarly-sized clusters

3. **Choosing K (Elbow Method)**
   - Plot inertia vs K
   - Look for "elbow" where gains diminish
   - Combine with silhouette score for validation

4. **Cluster Evaluation**
   - **Silhouette score**: Measures cluster separation (-1 to +1)
   - **Inertia**: Within-cluster sum of squares (lower is better)
   - **Davies-Bouldin**: Average similarity between clusters (lower is better)

5. **K-Means Limitations**
   - Assumes spherical clusters
   - Sensitive to initialization
   - Requires K in advance
   - Affected by outliers

6. **DBSCAN Clustering**
   - Density-based: clusters are dense regions
   - No need to specify K
   - Handles arbitrary shapes
   - Identifies outliers automatically

7. **DBSCAN Parameters**
   - **eps**: Maximum distance between neighbors
   - **min_samples**: Minimum points for dense region
   - Use K-distance plot to find optimal eps

8. **Algorithm Selection**
   - **K-Means**: Fast, spherical clusters, known K
   - **DBSCAN**: Arbitrary shapes, unknown K, outlier detection

### Best Practices

- **Always scale features** before clustering (especially for K-Means)
- **Use multiple evaluation metrics** (silhouette, inertia, visual inspection)
- **Try multiple K values** and compare results
- **Visualize clusters** when possible (2D or PCA projection)
- **Consider data characteristics** when choosing algorithm
- **Run K-Means multiple times** (n_init parameter) due to random initialization
- **Tune DBSCAN parameters** using K-distance plot

### Common Pitfalls to Avoid

- ❌ Forgetting to scale features
- ❌ Choosing K without validation (elbow method, silhouette)
- ❌ Using K-Means on non-spherical clusters
- ❌ Ignoring outliers in the data
- ❌ Trusting clustering results without domain knowledge
- ❌ Using inappropriate distance metrics

### What's Next

In **Module 13: Dimensionality Reduction (PCA, t-SNE)**, you'll learn:
- Why reduce dimensions (curse of dimensionality, visualization, speed)
- Principal Component Analysis (PCA) explained
- Choosing number of components (explained variance)
- t-SNE for 2D/3D visualization
- When to use PCA vs t-SNE
- PCA for preprocessing before machine learning

### Additional Resources

**Videos:**
- [StatQuest: K-Means Clustering](https://www.youtube.com/watch?v=4b5d3muPQmA)
- [StatQuest: DBSCAN](https://www.youtube.com/watch?v=RDZUdRSDOok)
- [Visualizing K-Means](https://www.youtube.com/watch?v=IuRb3y8qKX4)

**Documentation:**
- [scikit-learn Clustering Guide](https://scikit-learn.org/stable/modules/clustering.html)
- [KMeans API](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
- [DBSCAN API](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

**Articles:**
- [K-Means Clustering: Algorithm, Applications, Evaluation](https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a)
- [DBSCAN Python Example](https://towardsdatascience.com/dbscan-clustering-explained-97556a2ad556)